In [363]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [364]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import math as mt
import datetime as dt
from sklearn.cluster import KMeans
import random as ran
from sklearn.metrics.pairwise import euclidean_distances as euc
from functools import reduce

# 한글출력
matplotlib.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

from src import crs
from IPython.display import clear_output, Markdown, display

In [365]:
# 1. load_excel
data_path = "data/apt_1.xlsx"
xlsx = pd.read_excel(data_path, header=None,
                     skiprows=2, engine="openpyxl")

In [366]:
# 2. data preprocessing
p, m = crs.utils.data_preprocessing(xlsx)

m.set_index("month", inplace=True)
m.head()

/Users/formegusto/Desktop/idoformegusto/development/research/what-is-public-fee/src/crs/utils/data_preprocessing.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  datas_df[household_name] = household_df[col][3:].to_list()
/Users/formegusto/Desktop/idoformegusto/development/research/what-is-public-fee/src/crs/utils/data_preprocessing.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  month_usage_df[name] = [


,아파트1-101-1002,아파트1-101-104,아파트1-101-1102,아파트1-101-1202,아파트1-101-1302,아파트1-101-1401,아파트1-101-1403,아파트1-101-1502,아파트1-101-1504,아파트1-101-1602,...,아파트1-104-303,아파트1-104-404,아파트1-104-408,아파트1-104-503,아파트1-104-507,아파트1-104-606,아파트1-104-705,아파트1-104-708,아파트1-104-902,아파트1-104-905
month,,,,,,,,,,,,,,,,,,,,,
1,253,689,183,239,455,343,380,219,389,350,...,10,111,39,151,181,180,290,184,176,148
2,251,481,177,218,397,297,339,197,348,288,...,77,107,30,114,151,151,263,177,158,127
3,255,424,203,222,378,303,349,202,357,368,...,77,120,28,116,163,157,281,182,171,127
4,252,355,180,230,368,278,298,188,346,365,...,76,64,58,109,151,148,257,175,140,121
5,230,357,258,203,414,270,418,156,320,265,...,150,63,75,137,115,13,84,149,36,117


# 임의 가구 구성 셋팅

In [367]:
# _m = m.sample(n=50, axis=1)
# _m = m[m.columns[:50]]

_m = m.copy()

# 초기 정보 셋팅
- 월, 공용부 차지 percentage

In [369]:
_month = 1

month_df = pd.DataFrame(_m.loc[_month])
month_df.reset_index(inplace=True)

month_df.columns = ['name', 'usage (kWh)']

households_kWh = month_df['usage (kWh)'].sum()
households_count = len(month_df)
PUBLIC_PERCENTAGE = 30
APT = round((households_kWh * 100) / (100 - PUBLIC_PERCENTAGE))

APT_mean = round(APT / households_count)
households_mean = round(month_df['usage (kWh)'].mean())

display(Markdown(
    "**설정 월은 {}월 입니다.**<br/>".format(_month) +
    "• 가구 수 : {}가구<br/>".format(households_count) +
    "• APT 총 사용량 : {}kWh (평균사용량:{}kWh)<br/>".format(format(APT,","),
                                                   APT_mean) +
    "• 세대부 총 사용량 : {}kWh (평균사용량:{}kWh)".format(format(households_kWh, ","),
                                              households_mean)
))

month_df.head()

**설정 월은 1월 입니다.**<br/>• 가구 수 : 108가구<br/>• APT 총 사용량 : 44,434kWh (평균사용량:411kWh)<br/>• 세대부 총 사용량 : 31,104kWh (평균사용량:288kWh)

,name,usage (kWh)
0,아파트1-101-1002,253
1,아파트1-101-104,689
2,아파트1-101-1102,183
3,아파트1-101-1202,239
4,아파트1-101-1302,455


# 청구서 셋팅

In [370]:
calc = crs.models.ManagementOffice(
        month=_month,
        households=month_df,
        APT=APT,
        contract="단일계약"
    )
fee = calc.fee
fee

[[0, 200, 730, 73.3], [201, 400, 1260, 142.3], [401, inf, 6060, 210.6]]

# 기본요금 예측

## 누진단계 셋팅

In [371]:
nugin_steps = [_[0] for _ in fee]
nugin_basic = [_[2] for _ in fee]

In [372]:
nugin_steps

[0, 201, 401]

## 누진단계 분할

In [373]:
nugin_dist = pd.DataFrame(columns=['가구 수'])

for step, _f in enumerate(fee):
    now_kWh = _f[0]
    if step < 2:
        next_kWh = fee[step + 1][0]
        count = len(month_df[(month_df['usage (kWh)'] >= now_kWh) &\
                             (month_df['usage (kWh)'] < next_kWh)])
        nugin_dist.loc['{} 단계'.format(step + 1)] = [count]
                    
    else:
        count = len(month_df[month_df['usage (kWh)'] >= now_kWh])
        nugin_dist.loc['{} 단계'.format(step + 1)] = [count]
        
nugin_dist.head()

,가구 수
1 단계,20
2 단계,73
3 단계,15


## 수거액 산정

In [374]:
collect_basic = list()

for step,count in enumerate(nugin_dist['가구 수'].values):
    basic = fee[step][2]
    
    collect_basic.append(basic * count)
    
nugin_dist['수거액'] = collect_basic

nugin_dist.head()

,가구 수,수거액
1 단계,20,14600
2 단계,73,91980
3 단계,15,90900


## APT 누진 단계 확인 및 기본요금 확인

In [345]:
print("현재 APT 평균사용량은 {}kWh 입니다.".format(APT_mean))
print("요금표 {}에 근거하여,".format(fee))

for step, _f in enumerate(fee):
    if APT_mean < _f[1]:
        apt_step = step
        apt_basic = _f[2] * households_count
        break
        
print("\n현재 APT 평균사용량의 누진단계는 {}단계 입니다.".format(apt_step + 1))
print("{} 기본요금이 적용되어, 기본요금은 {} * {}가구 수 = {}원 입니다.".format(
    format(fee[apt_step][2], ","),
    format(fee[apt_step][2], ","),
    households_count,
    format(apt_basic, ",")
))

현재 APT 평균사용량은 473kWh 입니다.
요금표 [[0, 200, 730, 73.3], [201, 400, 1260, 142.3], [401, inf, 6060, 210.6]]에 근거하여,

현재 APT 평균사용량의 누진단계는 3단계 입니다.
6,060 기본요금이 적용되어, 기본요금은 6,060 * 50가구 수 = 303,000원 입니다.


## 공용부 기본요금 산출
- APT 기본요금 - 세대부 수거액

In [346]:
households_basic = nugin_dist['수거액'].sum()
public_basic = apt_basic - households_basic

print("APT 기본요금 : {}원".format(format(apt_basic, ",")))
print("세대부 기본요금 : {}원".format(format(households_basic, ",")))
print("공용부 기본요금 : {}원".format(format(public_basic, ",")))

APT 기본요금 : 303,000원
세대부 기본요금 : 114,210원
공용부 기본요금 : 188,790원


# 전력량 요금 예측

## 누진단계별 수거해야 하는 사용량

In [347]:
col_list = list()
_usage = households_mean
for step, _f in enumerate(fee):
    collect_rate = _f[1] - _f[0]

    ## 2단계일 경우
    if step == 1:
        collect_rate += 1

    if households_mean < _f[1]:
        if (step == 0) or (step == 1):
            _col = _usage % collect_rate

        else:
            _col = _usage
    else:
        _col = collect_rate

    if _usage == 0:
        break
    else:
        col_list.append(_col * households_count)
        print(_col)
        _usage -= _col

apt_col = pd.DataFrame(col_list, columns=['수거 필요 액수'])
apt_col.index = pd.Series(range(1, len(col_list) + 1), name="누진 단계")

apt_col

200
131


,수거 필요 액수
누진 단계,
1,10000
2,6550


## 가구별 누진 단계 수거 사용량

In [348]:
month_df.head(6)

,name,usage (kWh)
0,아파트1-101-1002,253
1,아파트1-101-104,689
2,아파트1-101-1102,183
3,아파트1-101-1202,239
4,아파트1-101-1302,455
5,아파트1-101-1401,343


In [349]:
# nugin elec rate collect
nerc = pd.DataFrame(columns=['가구명', '누진단계', '수거액'])

for idx, row in month_df.head(100).iterrows():
    name, usage = row
    
    col_list = list()
    _usage = usage
    for step, _f in enumerate(fee):
        collect_rate = _f[1] - _f[0]
            
        ## 2단계일 경우
        if step == 1:
            collect_rate += 1

        if usage < _f[1]:
            if (step == 0) or (step == 1):
                _col = _usage % collect_rate
                
            else:
                _col = _usage
        else:
            _col = collect_rate
            
        if _usage == 0:
            break
        else:
            col_list.append(_col)
            _usage -= _col
            
    for step,col in enumerate(col_list):
        nerc = nerc.append({
            "가구명": name,
            "누진단계": step + 1,
            "수거액": col
        }, ignore_index=True)
        
nerc = nerc.pivot_table(index=['누진단계'], values=['수거액'], aggfunc='sum')
nerc

,수거액
누진단계,
1,9868
2,5695
3,989


### 가구 수거액 확인

In [350]:
col_pd = pd.concat([apt_col, nerc], axis=1).fillna(0).astype("int")
col_pd['오차'] = col_pd['수거 필요 액수'] - col_pd['수거액']

col_pd

,수거 필요 액수,수거액,오차
1,10000,9868,132
2,6550,5695,855
3,0,989,-989


## 공용부 요금 시작 지점 확인

In [351]:
public_start = 0

for step, row in col_pd.iterrows():
    _f = fee[step - 1]
    rate = row['오차']
    
    public_start += (_f[3] * rate)
    
public_start = round(public_start)

print("공용부 시작 전력량 요금은 {}원 입니다.".format(format(public_start, ",")))

공용부 시작 전력량 요금은 -76,941원 입니다.


## 공용부 시작 누진 단계 확인

In [352]:
# 복구 로직
APT_mean = round(APT / households_count)
households_mean = round(month_df['usage (kWh)'].mean())

In [353]:
col_list = list()
_usage = APT_mean
for step, _f in enumerate(fee):
    collect_rate = _f[1] - _f[0]

    ## 2단계일 경우
    if step == 1:
        collect_rate += 1

    if APT_mean < _f[1]:
        if (step == 0) or (step == 1):
            _col = _usage % collect_rate
        else:
            _col = _usage
    else:
        _col = collect_rate

    if _usage == 0:
        break
    else:
        col_list.append(_col)
        _usage -= _col
        
_usage = households_mean
for idx,col in enumerate(col_list):
    if _usage < col:
        col_list[idx] -= _usage
        _usage -= _usage
    else:
        _usage -= col_list[idx]
        col_list[idx] = 0
        
    if _usage == 0:
        break
        
elec_rate_start = np.where(np.array(col_list) != 0)[0][0]
print("공용부 전력량 요금은 {}단계부터 시작합니다. 해당 단계의 나머지 사용량은 {}kWh입니다.".format(
    elec_rate_start + 1,
    col_list[elec_rate_start]
))

공용부 전력량 요금은 2단계부터 시작합니다. 해당 단계의 나머지 사용량은 69kWh입니다.


In [354]:
public_elec_rate = 0

for step,col in enumerate(col_list):
    _f = fee[step]
    public_elec_rate += _f[3] * col * households_count

public_elec_rate = round(public_elec_rate) + public_start
print("공용부 전력량 요금: {}원".format(format(public_elec_rate, ",")))

공용부 전력량 요금: 1,182,684원


In [355]:
public_env = round((APT - households_kWh) * 5.3)
public_fuel = round((APT - households_kWh) * 3)

print("공용부 기후 환경 요금 : {}원".format(format(public_env, ",")))
print("공용부 연료비 조정액 : {}원".format(format(public_fuel, ",")))

공용부 기후 환경 요금 : 37,598원
공용부 연료비 조정액 : 21,282원


In [356]:
public_elec_bill = public_basic + public_elec_rate + public_env - public_fuel
households_guarantees = sum([_.guarantee for _ in calc.households])

public_elec_bill += households_guarantees

print("공용부 전기요금계 : {}원".format(format(public_elec_bill, ",")))

공용부 전기요금계 : 1,395,290원


In [359]:
households_guarantees

7500

In [357]:
public_vat = round(public_elec_bill * 0.1)
public_fund = round(public_elec_bill * 0.037)

public_bill = public_elec_bill + public_vat + public_fund
print("공용부 예측 최종청구금액 : {}원".format(format(public_bill, ",")))

공용부 예측 최종청구금액 : 1,586,445원


In [358]:
calc.public_bill

1586890